# Importing ORACC Data from corpus.json
by Niek Veldhuis
UC Berkeley

March-June 2017

# TODO
* check that COFs are treated properly
* check that lines that continue into the next line (as in bilinguals) are captured completely. Such lines are indicated in the json by the the addition of 'l' (lower case L) to the reference (.ref).
* add definition of fields to list in Introduction

# Note
Currently the code will fetch a large zip file from ORACC, download it, extract certain files from the zip file and parse those. The zip file contains all data that belong to an ORACC project or sub-project. Since one may run this notebook several times for collecting data from the same project, this may not be the best process (the download will take place every time). Move the download process to a separate notebook, preceding the current one.


# Introduction

Purpose of the code is to download [ORACC](http://oracc.org) (Open Richly Annotated Cuneiform Corpus) JSON files that contain textual data and produce a `.csv` file in the directory `data/raw` with the relevant data for use in the phylogenetics project. The JSON files contain all the transliteration and lemmatization data of an ORACC project (metadata are made available in a separate `.json` file). For an introduction to the various ORACC JSON files see the [ORACC Open Data](http://oracc.org/doc/opendata) page.

The resulting data file includes various elements of the ORACC data structure. The current code will output a file with the following fields: 

* id_line
* label
* lemma
* base
* extent
* scope

The fields `extent` and `scope` capture the number of missing lines or columns.

The selection of fields may be adjusted with standard `Pandas` functions.

## Notes
The current version of the script works with the `ijson` library. Documentation for [ijson](https://www.dataquest.io/blog/python-json-tutorial/), unfortunately, is extremely brief. 

This notebook is written for **Python 3.5** with **Pandas 0.19** and **ijson 2.3**.

The notebook was written for the [Digital Humanities Phylogenetics](https://github.com/ErinBecker/digital-humanities-phylogenetics) project with Erin Becker of [Data Carpentry](http://www.datacarpentry.org). The particular data selection and data manipulation performed in this notebook are inspired by the needs of that project (for instance, non-Sumerian words are filtered out). It should be fairly easy to adapt the notebook to the purposes of any other project that wishes to use [ORACC](http://oracc.org) data.

## Licensing
This notebook may be downloaded, used and adapted without any restrictions.

In [1]:
import pandas as pd   
import ijson
import urllib.request
import re
import zipfile
import tqdm

# Input List of Text IDs
Identify a list of text IDs (P, Q, and X numbers) in the directory `data/text_ids`. The IDs are six-digit P, Q, or X numbers preceded by a project abbreviation in the format 'PROJECT/P######' or 'PROJECT/SUBPROJECT/Q######'. For example:
* dcclt/P117395
* etcsri/Q001203
* rinap/rinap1/Q003421

The list should be created with a flat text editor such as Textedit or Emacs (not in a word processor such as MS Word), and the filename should end in `.txt`.

In [2]:
filename = input('Filename: ')

Filename: Q39_par.txt


In [3]:
textids = '../data/text_ids/' + filename
with open(textids, 'r') as f:
    pqxnos = f.readlines()
pqxnos = [x.strip() for x in pqxnos]
projects = [x[:-8] for x in pqxnos]
projects = list(set(projects))
pqxnos[:5], projects

(['dcclt/Q000039',
  'dcclt/P117395',
  'dcclt/P117404',
  'dcclt/P128345',
  'dcclt/P224980'],
 ['dcclt'])

# Download `json.zip`
For each project from which files are to be processed download the entire project (all the json files) in `http://oracc.museum.upenn.edu/PROJECT/json.zip`. For larger projects (such as [DCCLT](http://oracc.org/dcclt)) the `json.zip` may be 25Mb or more. Downloading may take some time and it may be necessary to chunk the downloading process. For the chunking code see instructions and explanations [here](https://www.smallsurething.com/how-to-read-a-file-properly-in-python/).

Although downloading the entire zip file is time consuming, it will make processing the individual files much more efficient and the code is less likely to break due to interruption in connectivity.

In [4]:
for project in projects:
    url = "http://oracc.museum.upenn.edu/" + project + "/json.zip"
    file = '../data/json/' + project.replace('/', '_') + '_json.zip'
    print("Downloading " + url + " saving as " + file)
    response = urllib.request.urlopen(url)
    CHUNK = 16 * 1024
    with open(file, 'wb') as f:
        for chunk in tqdm.tqdm(iter(lambda: response.read(CHUNK), b'')):
            f.write(chunk)

1541it [03:07,  9.28it/s]


# Extract
Extract from the `json.zip` the individual `.json` files enumerated in the list of text IDs. All files are extracted to a directory called `data/json/corpusjson`. If a list of text IDs has the same P number multiple times (e.g. if editions of the same text exist in multiple projects), the file will be overwritten and only one instance of that P number will be available.

In [5]:
target_dir = '../data/json'
for no in pqxnos:
    project = no[:-8]
    pno = no[-7:]
    zip_file = "../data/json/" + project.replace('/', '_') + "_json.zip"
    with zipfile.ZipFile(zip_file,"r") as zip_ref:
        try:
            file = 'corpusjson/' + pno + '.json'
            zip_ref.extract(file, target_dir)
        except:
            print(no + ' is not available')

# Parse
The function `oraccjsonparser()` takes one argument (the name of the `.json` file). It looks for the prefix `textid` to retrieve the six-digit P, Q, or X number of the text artifact. Parsing the file sequentially the code looks for the places where a line starts (`'.type' = 'line-start'`) and where a word starts (`'.node' = 'l'`, where `l` is for "lemma"). At each level the code will retrieve the relevant data and create a list of dictionaries where each dictionary represents a single word. 

Words not only include lemmatized words, but also unlemmatized and unlemmatizable words (such as breaks).

The resulting dictionary includes keys such as `lang` (for language), `guideword`, `sense`, etc. - all the elements that define an [ORACC](http://oracc.org) signature. The dictionary also includes the keys `id_line` (a sequential number for each line or line-like element in a text) and `id_word` (a sequential number for each word in a line) that allow the user to reassemble a text in the original word and line order.

In [6]:
def oraccjsonparser(file):
    filename = '../data/json/corpusjson/' + file +'.json'
    with open(filename, 'r') as d:
        parser = ijson.parse(d)
        word_l = []
        word_d = {}
        line_start = False
        word_start = False
        nonx = False
        for prefix, event, value in parser: #prefix, event, and value are the standard
                                            #elements of a JSON entry recognized by ijson
            if prefix == 'textid':
                id_text = value
            if prefix.endswith('.type'):
                if value == 'line-start':
                    line_start = True
                else:
                    line_start = False
            if line_start:
                if prefix.endswith('.ref') and not word_start:
                    id_line = value # id_line is a reference number for a line
                                    # that includes the id_text (e.g. P123456.49)
                if prefix.endswith('.label'):
                    label = value   # label is a human-readable line number of the format
                                    # o ii 24' (obverse column 2 line 24')
            if prefix.endswith('node'):
                if value == 'l':
                    word_start = True
                    if not word_d == {}:
                        word_l.append(word_d) # append the previous word to the list
                    word_d = {}               # and start a new dictionary
                    word_d['id_text'] = id_text # provide each word with appropriate 
                    word_d['id_line'] = id_line # text and line-ID
                    word_d['label'] = label     # and the line label.
                else:
                    word_start = False
            if word_start:
                if prefix.endswith('.ref'):
                    word_d['id_word'] = value
                if prefix.endswith('.sig'):
                    word_d['signature'] = value
                if '.f.' in prefix:
                    category = re.sub('.*\.', '', prefix) # get element after the last dot of the prefix
                    word_d[category] = value # copy each element into the dictionary
            if prefix.endswith('.type'):
                if value == 'nonx':
                    nonx = True
                else:
                    nonx = False
            if nonx:                         # this captures so-called $-lines with information
                if prefix.endswith('.ref'):  # about number of broken lines/columns.
                    id_line = value          # $-lines have their own id_line.
                if prefix.endswith('.strict'):
                    if value == '1':           # select only 'strict' $ lines
                        if not word_d == {}:
                            word_l.append(word_d)
                        word_d = {}
                        word_d['id_line'] = id_line
                        word_d['id_text'] = id_text
                    else:
                        nonx = False
                if prefix.endswith('.extent'): # capture the three elements of strict $ lines
                    word_d['extent'] = value   # namely extent, scope, and state.
                if prefix.endswith('.scope'):
                    word_d['scope'] = value
                if prefix.endswith('.state'):
                    word_d['state'] = value

    word_l.append(word_d)  # make sure that the last word is captured, too.
    return(word_l) # return a list of dictionaries, where each entry (dictionary) in
                   # the list represents a word.

# Call the Parser Function for Each Textid

In [7]:
word_l = []
for no in tqdm.tqdm(pqxnos):
    id_text = no[-7:]
    try:
        word_l.extend(oraccjsonparser(id_text))
    except:
        print(no + ' not available')

100%|██████████| 136/136 [00:08<00:00, 16.49it/s]


# Transform the Data into a DataFrame

If a text has no breakage information in the form of `$ 1 line broken` (etc.) the fields `extent`, `scope`, and `state` do not exist. The fields `extent` and `scope` are referenced in the code below. After creating the dataframe the existence of these two fields is checked - if they do not exist, empty columns are created.

In [8]:
words = pd.DataFrame(word_l)
if not 'extent' in words.columns:
    words['extent'] = ''
if not 'scope' in words.columns:
    words['scope'] = ''
words.head()

,base,cf,cont,epos,extent,form,gw,id_line,id_text,id_word,label,lang,morph,norm,norm0,pos,scope,sense,signature,state
0,{ŋeš}taškarin,taškarin,NaN,N,NaN,{ŋeš}taškarin,boxwood,Q000039.1,Q000039,Q000039.1.1,1,sux,~,NaN,taškarin,N,NaN,"box tree, boxwood",@dcclt%sux:{ŋeš}taškarin=taškarin[boxwood//box...,NaN
1,{ŋeš}esi,esi,NaN,N,NaN,{ŋeš}esi,tree,Q000039.2,Q000039,Q000039.2.1,2,sux,~,NaN,esi,N,NaN,ebony,@dcclt%sux:{ŋeš}esi=esi[tree//ebony]N'N$esi/{ŋ...,NaN
2,ŋeš-nu₁₁,ŋešnu,NaN,N,NaN,{ŋeš}nu₁₁,tree,Q000039.3,Q000039,Q000039.3.1,3,sux,~,NaN,ŋešnu,N,NaN,tree,@dcclt%sux:{ŋeš}nu₁₁=ŋešnu[tree//tree]N'N$ŋešn...,NaN
3,{ŋeš}ha-lu-ub₂,halub,NaN,N,NaN,{ŋeš}ha-lu-ub₂,tree,Q000039.4,Q000039,Q000039.4.1,4,sux,~,NaN,halub,N,NaN,tree,@dcclt%sux:{ŋeš}ha-lu-ub₂=halub[tree//tree]N'N...,NaN
4,{ŋeš}šag₄-kal,šagkal,NaN,N,NaN,{ŋeš}šag₄-kal,tree,Q000039.5,Q000039,Q000039.5.1,5,sux,~,NaN,šagkal,N,NaN,tree,@dcclt%sux:{ŋeš}šag₄-kal=šagkal[tree//tree]N'N...,NaN


# Remove Spaces and Commas from Guide Word and Sense
Spaces in Guide Word and Sense may cause trouble in computational methods in tokenization, or when saved in Comma Separated Values format. All spaces and commas are replaced by hyphens or nothing, respectively.

In [9]:
words = words.fillna('') # first replace Missing Values by empty string
words['sense'] = [x.replace(' ', '-') for x in words['sense']]
words['sense'] = [x.replace(',', '') for x in words['sense']]
words['gw'] = [x.replace(' ', '-') for x in words['gw']]
words['gw'] = [x.replace(',', '') for x in words['gw']]

The columns in the resulting DataFrame correspond to the elements of a full [ORACC](http://oracc.org) signature, plus information about text, line, and word ids:
* base (Sumerian only)
* cf (Citation Form)
* cont (continuation of the base; Sumerian only)
* epos (Effective Part of Speech)
* form (transliteration, omitting all flags such as indication of breakage)
* gw (Guide Word: main or first translation in standard dictionary)
* id_line (a line ID that begins with the six-digit P, Q, or X number of the text)
* id_text (six-digit P, Q, or X number)
* id_word (word ID that begins with the ID number of the line)
* label (traditional line number in the form o ii 2' (obverse column 2 line 2'), etc.)
* lang (language code, including sux, sux-x-emegir, sux-x-emesal, akk, akk-x-stdbab, etc)
* morph (Morphology; Sumerian only)
* norm (Normalization: Akkadian)
* norm0 (Normalization: Sumerian)
* pos (Part of Speech)
* sense (contextual meaning)
* signature (full ORACC signature)

Not all data elements (columns) are available for all words. Sumerian words never have a `norm`, Akkadian words do not have `norm0`, `base`, `cont`, or `morph`. Most data elements are only present when the word is lemmatized; only `lang`, `form`, `pos`, `id_word`, `id_line`, and `id_text` should always be there. An unlemmatized word has `pos` 'X' (for unknown). Broken words have `pos` 'u' (for 'unlemmatizable).

# Manipulate
The columns may be manipulated with standard Pandas methods to create the desired output. By way of example, the following code will create a column `lemma` with the format **cf[gw]pos** (for instance **lugal[king]N**). For words that have no lemmatization, `lemma` equals `form`. Only Sumerian words are allowed (and thus `lang` can be omitted) and in addition to the column `lemma` the column `base` is preserved; words that have no lemmatization take `form` as their base. Words and bases are concatenated to lines.

## Remove  non-Sumerian words

In [10]:
lang = ['sux', ''] # note that 'lang' is empty in entries that indicate damage
words = words.loc[words['lang'].str[:3].isin(lang)].reset_index()

## Create Lemma Column and Adjust Base

In [11]:
words['lemma'] = words['cf'] # first element of lemma is the citation form
words['lemma'] = [words['lemma'][i] + '[' + words['gw'][i] 
                     + ']' + words['pos'][i] 
                     if not words['lemma'][i] == '' 
                     else words['form'][i] +'[NA]NA' for i in range(len(words))]
words['lemma'] = [lemma if not lemma == '[NA]NA' else '' for lemma in words['lemma'] ]
words['base'] = [words['base'][i] if not words['base'][i] == '' 
                 or words['label'][i] == '' else words['form'][i] 
                 for i in range(len(words))]

## Group by Line

In [12]:
df = words.groupby([words['id_line'], words['label']]).agg({
        'lemma': ' '.join,
        'base': ' '.join,
        'extent': ''.join, 
        'scope': ''.join
    }).reset_index()
df        

,id_line,label,lemma,base,extent,scope
0,P117395.2,o 1,ŋešed[key]N,{ŋeš}e₃-a,,
1,P117395.3,o 2,pakud[~tree]N,{ŋeš}pa-kud,,
2,P117395.4,o 3,raba[clamp]N,{ŋeš}raba,,
3,P117404.2,o 1,ig[door]N eren[cedar]N,{ŋeš}ig {ŋeš}eren,,
4,P117404.3,o 2,ig[door]N dib[board]N,{ŋeš}ig dib,,
5,P117404.4,o 3,ig[door]N i[oil]N,{ŋeš}ig i₃,,
6,P128345.2,o 1,garig[comb]N siki[hair]N,{ŋeš}ga-rig₂ siki,,
7,P128345.3,o 2,garig[comb]N siki-siki[NA]NA,{ŋeš}ga-rig₂ siki-siki,,
8,P128345.4,o 3,garig[comb]N saŋdu[head]N,{ŋeš}ga-rig₂ saŋ-du,,
9,P224980.4,o i 1,gigir[chariot]N,{ŋeš}gigir,,


# Create fields `id_text` and `line`
The field `id_text` is the P-number of the document and is equal to the first seven characters of `id_line`. The field `line` is an artificial line counter (integer) that is used to keep things in the right order. It derives from `id_line` by keeping eveything after the last dot (note that the regular expression `.+\.` is greedy and will match the longest possible string ending in a dot).

In [13]:
pd.options.mode.chained_assignment = None # this prevents Pandas from issuing warnings about chained assignments
df['id_text'] = df['id_line'].str[:7]
df['line'] = [int(re.sub('.+\.', '', line)) for line in df['id_line']] #create a line number for sorting
df = df.sort_values(['id_text', 'line']).reset_index(drop=True)
df

,id_line,label,lemma,base,extent,scope,id_text,line
0,P117395.2,o 1,ŋešed[key]N,{ŋeš}e₃-a,,,P117395,2
1,P117395.3,o 2,pakud[~tree]N,{ŋeš}pa-kud,,,P117395,3
2,P117395.4,o 3,raba[clamp]N,{ŋeš}raba,,,P117395,4
3,P117404.2,o 1,ig[door]N eren[cedar]N,{ŋeš}ig {ŋeš}eren,,,P117404,2
4,P117404.3,o 2,ig[door]N dib[board]N,{ŋeš}ig dib,,,P117404,3
5,P117404.4,o 3,ig[door]N i[oil]N,{ŋeš}ig i₃,,,P117404,4
6,P128345.2,o 1,garig[comb]N siki[hair]N,{ŋeš}ga-rig₂ siki,,,P128345,2
7,P128345.3,o 2,garig[comb]N siki-siki[NA]NA,{ŋeš}ga-rig₂ siki-siki,,,P128345,3
8,P128345.4,o 3,garig[comb]N saŋdu[head]N,{ŋeš}ga-rig₂ saŋ-du,,,P128345,4
9,P224980.4,o i 1,gigir[chariot]N,{ŋeš}gigir,,,P224980,4


## Save in CSV Format

In [14]:
filename = '../data/raw/' + filename[:-4] + '.csv'
print('saving ' + filename)
with open(filename, 'w') as w:
    df.to_csv(w, encoding='utf8')

saving ../data/raw/Q39_par.csv
